This notebook updates the table 1h_prices_last_enriched  
1h_prices_last_enriched stores 1 record per item with the last known price (and maybe volume?)

In [0]:
import pyspark.sql.functions as sf
from delta.tables import DeltaTable

In [0]:
# TODO replace the two code cells below with references to runescape.02_silver.1h_prices_enriched
# Read data from 'runescape.02_silver.1h_prices_cleansed'
df_1h_prices_cleansed = spark.read.table("runescape.02_silver.1h_prices_cleansed")

In [0]:
# Load item mapping data from the Silver Layer
df_item_mapping = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
# enrich price data with map data
df_1h_prices_enriched = df_1h_prices_cleansed.join(df_item_mapping, "id")

In [0]:
# create df with only the latest 1h_prices
# get latest time for each id
df_latest_time = df_1h_prices_enriched.groupBy("id").max("time").withColumnRenamed("max(time)", "time")
df_1h_prices_last_enriched = df_1h_prices_enriched.join(df_latest_time, ["id", "time"])

In [0]:
# Upsert data into "runescape.02_silver.1h_prices_last_enriched"

targetDF = DeltaTable.forName(spark, "runescape.02_silver.1h_prices_last_enriched")
dfUpdates = df_1h_prices_last_enriched

(targetDF.alias("t")
  .merge(
    source = dfUpdates.alias("s"),
    condition = "t.id = s.id")
  .whenMatchedUpdateAll()
  .execute())